## ОПРЕДЕЛЕНИЕ СЛУЧАЕВ ПАДЕНИЯ ЧЕЛОВЕКА С ПОМОЩЬЮ ТЕХНОЛОГИЙ КОМПЬЮТЕРНОГО ЗРЕНИЯ ##
**Описание проекта**
Проект **[Detect-fall](https://github.com/tickflag/Detect-fall)** представляет собой систему, разработанную для определения случаев падения человека с использованием технологий компьютерного зрения.

**Авторы**
- Львов Станислав
- Дмитриева Настя
- Иванов Павел
- Порфирьев Кирилл

> Код проекта доступен на [GitHub](https://github.com/tickflag/Detect-fall)


# Installation

In [ ]:
pip install ultralytics opencv-python numpy

# Settings

In [ ]:
import torch
settings = {
    'device': '0', #device to run on, i.e. cuda device='0/1/2/3' or device='cpu'
    'source': 'datasets/our-images/04.mp4', #Source directory for videos
    'pose_model': 'yolov8s-pose.pt' #Source directory for yolov8-pose model
}
if not torch.cuda.is_available():
    settings['device'] = 'cpu'

# Imports

In [ ]:
from ultralytics import YOLO
import cv2
import datetime
from tkinter import messagebox
import numpy as np
import math 
import threading

# DETECT-FALL

- Обработка изображений с помощью **[yolov8](https://docs.ultralytics.com)**

In [ ]:
model_pose = YOLO(settings['pose_model'])

def yolo_pose(source):
    return model_pose.predict(
        source=source, save=True, conf=0.3, device=0
    )

- Расчитываем лежит ли человек

In [ ]:
def detect_fall(source):
    peoples = []
    pose = yolo_pose(source=source)

    for res1 in pose:
        for element1 in res1:
            box = element1.boxes.cpu().numpy().xyxy[0]
            pose = element1.keypoints.cpu().numpy().xy[0]
            speed = xy_speed(box)
            chance = predict_possition_pose(pose=pose)
            peoples.append([chance, speed])
    
    return peoples

def xy_speed(xy): #TODO Сделать алгоритм высчитывания угловой скорости
    return 0

def predict_possition_pose(pose):

    left_shoulder = pose[5]
    right_shoulder = pose[6]
    left_hip = pose[11]
    right_hip = pose[12]
    
    head = pose[0]

    if [head[0], head[1]] == [0.0, 0.0]:
        for head_element in range(1, 4):
            if pose[head_element][0] != 0.0 and pose[head_element][1] != 0.0: 
                head = pose[head_element] 
                break
    
    angles = first_arg(left_shoulder=left_shoulder, left_hip=left_hip,
            right_shoulder=right_shoulder, right_hip=right_hip)
    btoa = second_arg(left_shoulder=left_shoulder, left_hip=left_hip, 
            right_shoulder=right_shoulder, right_hip=right_hip)
    head_pose = third_arg(head=head, left_shoulder=left_shoulder, right_shoulder=right_shoulder)

    first = first_coef(angles=angles)
    second = second_coef(btoa=btoa)
    third = third_coef(head_pose=head_pose)

    return (first + second + third) / 3

def first_arg(left_shoulder, left_hip, right_shoulder, right_hip):

    alpha = None
    betta = None

    if ([left_shoulder[0], left_shoulder[1]] != [0.0, 0.0]) and ([left_hip[0], left_hip[1]] != [0.0, 0.0]):
        alpha = get_angle(left_shoulder, left_hip)
    if ([right_shoulder[0], right_shoulder[1]] != [0.0, 0.0]) and ([right_hip[0], right_hip[1]] != [0.0, 0.0]):
        betta = get_angle(right_shoulder, right_hip)

    return [alpha, betta]

def get_angle(arg0, arg1):
    yy = abs(arg0[1] - arg1[1])
    xx = abs(arg0[0] - arg1[0])
    c = math.sqrt(math.pow(yy, 2) + math.pow(xx, 2))

    return (int(math.acos(yy / c) * 180 / math.pi))

def second_arg(left_shoulder, left_hip, right_shoulder, right_hip):

    c = None
    d = None

    if ([left_shoulder[0], left_shoulder[1]] != [0.0, 0.0]) and ([left_hip[0], left_hip[1]] != [0.0, 0.0]):
        c = get_angle(left_shoulder, left_hip)
    if ([right_shoulder[0], right_shoulder[1]] != [0.0, 0.0]) and ([right_hip[0], right_hip[1]] != [0.0, 0.0]):
        d = get_angle(right_shoulder, right_hip)

    return [c, d]

def get_(arg0, arg1):
    a = abs(arg0[0] - arg1[0])
    b = abs(arg0[1] - arg1[1])
    return b / a

def third_arg(head, left_shoulder, right_shoulder):

    is_above = above(head=head, left_shoulder=left_shoulder, right_shoulder=right_shoulder)
    is_right = rigth(head=head, left_shoulder=left_shoulder, right_shoulder=right_shoulder)
    is_left = left(head=head, left_shoulder=left_shoulder, right_shoulder=right_shoulder)

    if is_above:
        return False
    elif is_right or is_left:
        return True
    else:
        return 2

def above(head, left_shoulder, right_shoulder):
    return (head[1] < left_shoulder[1]) and (head[1] < right_shoulder[1])

def rigth(head, left_shoulder, right_shoulder):
    first = ((head[0] >= left_shoulder[0] and head[0] >= right_shoulder[0]) and \
        (right_shoulder[1] <= head[1] and left_shoulder[1] >= head[1]))
    second = ((head[0] >= left_shoulder[0] and head[0] <= right_shoulder[0]) and \
        (right_shoulder[1] <= head[1] and left_shoulder[1] >= head[1]))
    
    return first or second

def left(head, left_shoulder, right_shoulder):
    first = ((head[0] <= left_shoulder[0] and head[0] <= right_shoulder[0]) and \
        (right_shoulder[1] >= head[1] and left_shoulder[1] <= head[1]))
    second = ((head[0] >= left_shoulder[0] and head[0] <= right_shoulder[0]) and \
        (right_shoulder[1] >= head[1] and left_shoulder[1] <= head[1]))
    
    return first or second

def first_coef(angles):
    try: first = angles[0] // 30 > 0 
    except: first = 0.25
    try: second = angles[1] // 30 > 0
    except: second = 0.25

    return (int(first) + int(second)) / 2

def second_coef(btoa):
    try: first = btoa[0] < 1
    except: first = 0.25
    try: second = btoa[1] < 1
    except: second = 0.25

    return (int(first) + int(second)) / 2

def third_coef(head_pose):
    return head_pose / 2

- Инициализация

In [ ]:
def process_frame(frame):
    cv2.imwrite('input.jpg', frame)
    peoples = detect_fall('input.jpg')
    print(peoples)
    return sum(1 for person in peoples if person[0] >= 0.5)

def main():
    full = datetime.datetime.now()
    all_frames = 0
    
    cv2.namedWindow("preview")
    vc = cv2.VideoCapture(settings['source'])
    
    if not vc.isOpened():
        print("Error: Couldn't open video source.")
        return 0
    
    requared_frames = 5
    frames = 0
    
    while True:
        success, frame = vc.read()
        
        if not success:
            print("Error: Couldn't read frame.")
            break

        lay_persons = process_frame(frame)

        if lay_persons > 0:
            frames += 1
        else:
            frames = 0

        if frames == requared_frames:
            print("Человек упал")
            print('-' * 50, '\n')
            frames = 0
            alarm_message()
    vc.release()
    print(f"time taken {datetime.datetime.now() - full}")

def alarm_message():
    messagebox.showwarning("Alarm", "Обнаружен упавший человек")


if __name__ == "__main__":
    main()
